In [1]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
from Bio.PDB import Polypeptide

In [2]:
import pandas as pd
GP = pd.read_csv('DF_GEMPRO.csv', index_col=0)
# forcing gene IDs to be read as strings
GP['m_gene_original'] = GP['m_gene_original'].astype(str)
GP['m_gene_entrez'] = GP['m_gene_entrez'].astype(str)
GP['m_gene_isoform'] = GP['m_gene_isoform'].astype(str)
gene_entrez_id=raw_input('Enter Gene Entrez ID:')
GEM_PRO_available_refseq = GP[GP.m_gene_entrez == gene_entrez_id]['u_refseq']
print "These are the refseq IDs compatible with our workflow"
pd.DataFrame(GEM_PRO_available_refseq)

Enter Gene Entrez ID:2271
These are the refseq IDs compatible with our workflow


,u_refseq
1223,NP_000134


In [3]:
GP = pd.read_csv('DF_GEMPRO.csv', index_col=0)
# forcing gene IDs to be read as strings
GP['m_gene_original'] = GP['m_gene_original'].astype(str)
GP['m_gene_entrez'] = GP['m_gene_entrez'].astype(str)
GP['m_gene_isoform'] = GP['m_gene_isoform'].astype(str)
# this searches for an ID and prints out which row it is in
#gene_id = raw_input("What is the uniprot ID?   ") # this can be modified to ask for gene original, entrez, uniprot, isoform id, refseq etc.
acc=raw_input('Enter the refseq ID (u_refseq) of the sequence you would like to download:   ')
GP[GP.u_refseq == acc]

Enter the refseq ID (u_refseq) of the sequence you would like to download:   NP_000134


,m_gene_original,m_gene_entrez,m_gene_isoform,u_uniprot_acc,u_isoform_id,u_refseq,u_ensp,u_seq_len,u_seq,u_reviewed,...,ssb_p_aln_coverage,ssb_p_percent_seq_ident,ssb_p_no_deletions_in_pdb,ssb_p_aln_coverage_sim,ssb_si_score,ssb_rez_score,ssb_raw_score,ssb_above_cutoffs,ssb_rank,ssb_best_file
1223,2271.1,2271,1,P07954,P07954-1,NP_000134,ENSP00000355518,510,MYRALRLLARSRPLVRAPAAALASAPGLGGAAVPSFWPPNAARMAS...,True,...,459,0.9,True,459,1.44396,1.141667,2.585627,True,1,3e04.pdb


In [4]:
# This extracts all chains present
#print "These chains are present in the pdb structure: %s" %(chain_strings)
#pdb_chain_choose = raw_input("Which chain are you interested in?   ")
chains_avail = GP[GP.u_refseq == acc].p_chains
chains_present = ""
for a in chains_avail:
    chains_present = a

# This automatically displays/chooses which chain to align as it is the "best"; a string of A,B,C is returned
best_pdb_chain = GP[GP.u_refseq == acc].p_chain_uniprot_map.values[0][2]

In [6]:
# load Biopython PDB packages

# PDBList to download PDBs
from Bio.PDB.PDBList import PDBList
pdbl = PDBList()

# PDBParser to load and work with files
from Bio.PDB.PDBParser import PDBParser
parser = PDBParser()

import urllib2
import uuid

pdb_name = raw_input("What is the pdb ID?   ")

# download pdb
pdb_file_path = pdbl.retrieve_pdb_file(pdb_name)

What is the pdb ID?   3e04
Structure exists: '/Users/LAURENCE/Desktop/Senior Design/e0/pdb3e04.ent' 


In [8]:
# we can put a raw input to name the structure as well
protein_name = raw_input("What is the name of the protein of interest?   ")
label_input = protein_name
pdb_structure = parser.get_structure(protein_name, pdb_file_path)

What is the name of the protein of interest?   FH


In [9]:
# get the ligands within this file for display
# from: http://stackoverflow.com/questions/25718201/remove-heteroatoms-from-pdb
ligands = []

for residue in pdb_structure.get_residues():
    tags = residue.get_full_id()
    # tags contains a tuple with (Structure ID, Model ID, Chain ID, (Residue ID))
    # Residue ID is a tuple with (*Hetero Field*, Residue ID, Insertion Code)

    # Thus you're interested in the Hetero Field, that is empty if the residue
    # is not a hetero atom or have some flag if it is (W for waters, H, etc.)
    if tags[3][0] != " " and tags[3][0] != "W":
        ligands.append(tags[3][0].split('_')[1].strip())
    else:
        continue
        
print(ligands)

['EDO', 'EDO', 'EDO']


In [10]:
print "These are the chains present in the structure:   " + chains_present
chain_display = raw_input("Would you like to display all chains (answer with yes or no)?   ")
if chain_display.upper() == 'YES':
    cnames_pv_var = ''
    structure_var = 'structure'
    chain_iso = ''
elif chain_display.upper() == 'NO':
    cnames = raw_input("Type in the chain you would like to display:   ")
    cnames_pv = "cname: '" + cnames + "'"
    cnames_pv_var = "var chain = structure.select({cname: '" + cnames + "'})"
    structure_var = 'chain'
else:
    print "not sure what's here yet..."

#print "These are the chains present in the structure:   " + chains_present
chain_display = raw_input("Would you like to display all chains (answer with yes or no)?   ")
if chain_display.upper() == 'YES':
    cnames_pv_var = ''
    structure_var = 'structure'
    chain_iso = ''
elif chain_display.upper() == 'NO':
    cnames = raw_input("Type in the chain you would like to display:   ")
    cnames_pv = "cname: '" + cnames + "'"
    cnames_pv_var = "var chain = structure.select({cname: '" + cnames + "'})"
    structure_var = 'chain'
else:
    print "not sure what's here yet..."
    
bind_site_avail = raw_input("Is there a binding or active site you would like to display (answer with yes or no)?   ")
if bind_site_avail.upper() == 'YES':
    chain_res_disp_choice = raw_input("Would you like to display the site on all chains (answer with yes or no)?   ")
    # print "These are the chains present in the structure:   " + chains_present
    if chain_res_disp_choice.upper() == 'YES':
        chain_iso = ''
        is_res = ''
    elif chain_res_disp_choice.upper() == 'NO':
        chain_res_disp_chain_choice = raw_input("Which chain would you like to display the site on?   ")
        is_res = ''
        chain_iso = "cname: '" + chain_res_disp_chain_choice + "'" + ", "
    else:
        print "tbd"
elif bind_site_avail.upper() == 'NO':
    site_start = 0
    site_end = 0
    rnums_script = ''
    is_res = '//'
else:
    print "tbd"

binding_site_list = raw_input("List the sequences of the active sites (ie 6, 7, 120-134):   ")
binding_site_list_strip = binding_site_list.replace(' ','')
mylist = binding_site_list_strip.split(',')
indexx = 0
numberz = ''
for number in mylist:
    if '-' not in mylist[indexx]:
        numberz += ', ' + mylist[indexx]
    if '-' in mylist[indexx]:
        indexxx = 0
        for everydigit in mylist[indexx]:
            if everydigit == '-':
                hyphen_pos = indexxx
            indexxx = indexxx + 1
        rnums_seq_script = mylist[indexx][0:hyphen_pos]
        num_counter = mylist[indexx][0:hyphen_pos]
        numberz += ', '+ num_counter
        for y in range(int(mylist[indexx][(hyphen_pos + 1):]) - int(mylist[indexx][0:hyphen_pos])):
            num_counter = str(int(num_counter) + 1)
            numberz += ', ' + str(num_counter)
    indexx = indexx + 1
tidied_output = numberz[2:]
rnums_script = "rnums : [" + tidied_output + "]"
#print rnums_script

These are the chains present in the structure:   ['A', 'C', 'B', 'D']
Would you like to display all chains (answer with yes or no)?   yes
Would you like to display all chains (answer with yes or no)?   yes
Is there a binding or active site you would like to display (answer with yes or no)?   yes
Would you like to display the site on all chains (answer with yes or no)?   yes
List the sequences of the active sites (ie 6, 7, 120-134):   25-60


In [11]:
class PDBViewer_options(object):
    '''
    Contributed by: Ali Ebrahim
    '''
    
    def __init__(self, f):
        self.pdb = open(f).read()

    def _repr_html_(self):
        div_id = str(uuid.uuid4())
        
        return """<div id="%s" style="width: 800px; height: 600px">
    <div>
    
        <!--testing static label-->
        <style>
            .static-label {
                position: absolute;
                background: #0000;
                text-align: right;
                z-index: 1;
                font-weight: bold;
                width: 800px;
            }
        </style>
        
        <script>
            require.config({
                paths: {
                    "pv": "//biasmv.github.io/pv/js/pv.min"
                }
            });
            
            require(["pv"], function(pv) {
                pdb = "%s";
                
                <!--append the static label to the parent-->
                var parent = document.getElementById('%s');
                var staticLabel = document.createElement('div');
                staticLabel.innerHTML = '%s';
                staticLabel.className = 'static-label';
                parent.appendChild(staticLabel);
                
                <!--load the structure-->
                structure = pv.io.pdb(pdb);
                
                // select a chain to display and see if user wants to only display one
                %s
                
                <!--choose atom to label-->
                var carbonAlpha = structure.atom("%s");
                
                // choose a ligand to color (later on), if want to see on both chains remove cname
                %s var residues = structure.select({%s %s});
                
                viewer = pv.Viewer(parent, {
                    width: '800',
                    height: '600',
                    antialias: true,
                    outline: true,
                    quality: 'medium',
                    style: 'hemilight',
                    background: 'white',
                    animateTime: 500,
                    selectionColor: '#f00'
                });
            
                
                <!--misc viewer functions-->
                viewer.fitParent();
                
                // add cartoon visualization
                viewer.cartoon('molecule', %s);
                
                // color the selected residues in red, and display as red lines
                %s viewer.spheres('residues', residues,  { color: pv.color.uniform('red') });
                
                // center on the structure
                viewer.centerOn(%s);
                
                <!--atom label options-->
                var options = {
                 fontSize : 16, fontColor: '#f22', backgroundAlpha : 0.4
                };
                
                <!--display the label-->
                viewer.label('label', carbonAlpha.qualifiedName(), carbonAlpha.pos(), options);
                
                <!--not sure how the auto zoom works-->
                viewer.autoZoom();
            });
        </script>
        """ % (div_id, self.pdb.replace("\n", "\\n"), div_id, protein_name, cnames_pv_var, label_input, is_res, chain_iso, rnums_script, structure_var, is_res, structure_var)

## Displaying all chains and highlighting on all binding sites

In [21]:
PDBViewer_options(pdb_file_path)

In [12]:
pdb_file_path2 = pdbl.retrieve_pdb_file(pdb_name)

Structure exists: '/Users/LAURENCE/Desktop/Senior Design/e0/pdb3e04.ent' 


In [13]:
print "These are the chains present in the structure:   " + chains_present
chain_display = raw_input("Would you like to display all chains (answer with yes or no)?   ")
if chain_display.upper() == 'YES':
    cnames_pv_var = ''
    structure_var = 'structure'
    chain_iso = ''
elif chain_display.upper() == 'NO':
    cnames = raw_input("Type in the chain you would like to display:   ")
    cnames_pv = "cname: '" + cnames + "'"
    cnames_pv_var = "var chain = structure.select({cname: '" + cnames + "'})"
    structure_var = 'chain'
else:
    print "not sure what's here yet..."

#print "These are the chains present in the structure:   " + chains_present
chain_display = raw_input("Would you like to display all chains (answer with yes or no)?   ")
if chain_display.upper() == 'YES':
    cnames_pv_var = ''
    structure_var = 'structure'
    chain_iso = ''
elif chain_display.upper() == 'NO':
    cnames = raw_input("Type in the chain you would like to display:   ")
    cnames_pv = "cname: '" + cnames + "'"
    cnames_pv_var = "var chain = structure.select({cname: '" + cnames + "'})"
    structure_var = 'chain'
else:
    print "not sure what's here yet..."
    
bind_site_avail = raw_input("Is there a binding or active site you would like to display (answer with yes or no)?   ")
if bind_site_avail.upper() == 'YES':
    chain_res_disp_choice = raw_input("Would you like to display the site on all chains (answer with yes or no)?   ")
    # print "These are the chains present in the structure:   " + chains_present
    if chain_res_disp_choice.upper() == 'YES':
        chain_iso = ''
        is_res = ''
    elif chain_res_disp_choice.upper() == 'NO':
        chain_res_disp_chain_choice = raw_input("Which chain would you like to display the site on?   ")
        is_res = ''
        chain_iso = "cname: '" + chain_res_disp_chain_choice + "'" + ", "
    else:
        print "tbd"
elif bind_site_avail.upper() == 'NO':
    site_start = 0
    site_end = 0
    rnums_script = ''
    is_res = '//'
else:
    print "tbd"

binding_site_list = raw_input("List the sequences of the active sites (ie 6, 7, 120-134):   ")
binding_site_list_strip = binding_site_list.replace(' ','')
mylist = binding_site_list_strip.split(',')
indexx = 0
numberz = ''
for number in mylist:
    if '-' not in mylist[indexx]:
        numberz += ', ' + mylist[indexx]
    if '-' in mylist[indexx]:
        indexxx = 0
        for everydigit in mylist[indexx]:
            if everydigit == '-':
                hyphen_pos = indexxx
            indexxx = indexxx + 1
        rnums_seq_script = mylist[indexx][0:hyphen_pos]
        num_counter = mylist[indexx][0:hyphen_pos]
        numberz += ', '+ num_counter
        for y in range(int(mylist[indexx][(hyphen_pos + 1):]) - int(mylist[indexx][0:hyphen_pos])):
            num_counter = str(int(num_counter) + 1)
            numberz += ', ' + str(num_counter)
    indexx = indexx + 1
tidied_output = numberz[2:]
rnums_script = "rnums : [" + tidied_output + "]"
#print rnums_script

These are the chains present in the structure:   ['A', 'C', 'B', 'D']
Would you like to display all chains (answer with yes or no)?   yes
Would you like to display all chains (answer with yes or no)?   yes
Is there a binding or active site you would like to display (answer with yes or no)?   yes
Would you like to display the site on all chains (answer with yes or no)?   no
Which chain would you like to display the site on?   D
List the sequences of the active sites (ie 6, 7, 120-134):   25-60


## Displaying all chains but highlighting on only one chain

In [14]:
PDBViewer_options(pdb_file_path2)

In [16]:
print "These are the chains present in the structure:   " + chains_present
chain_display = raw_input("Would you like to display all chains (answer with yes or no)?   ")
if chain_display.upper() == 'YES':
    cnames_pv_var = ''
    structure_var = 'structure'
    chain_iso = ''
elif chain_display.upper() == 'NO':
    cnames = raw_input("Type in the chain you would like to display:   ")
    cnames_pv = "cname: '" + cnames + "'"
    cnames_pv_var = "var chain = structure.select({cname: '" + cnames + "'})"
    structure_var = 'chain'
else:
    print "not sure what's here yet..."
    
bind_site_avail = raw_input("Is there a binding or active site you would like to display (answer with yes or no)?   ")
if bind_site_avail.upper() == 'YES':
    chain_res_disp_choice = raw_input("Would you like to display the site on all chains (answer with yes or no)?   ")
    # print "These are the chains present in the structure:   " + chains_present
    if chain_res_disp_choice.upper() == 'YES':
        chain_iso = ''
        is_res = ''
    elif chain_res_disp_choice.upper() == 'NO':
        chain_res_disp_chain_choice = raw_input("Which chain would you like to display the site on?   ")
        is_res = ''
        chain_iso = "cname: '" + chain_res_disp_chain_choice + "'" + ", "
    else:
        print "tbd"
elif bind_site_avail.upper() == 'NO':
    site_start = 0
    site_end = 0
    rnums_script = ''
    is_res = '//'
else:
    print "tbd"

binding_site_list = raw_input("List the sequences of the active sites (ie 6, 7, 120-134):   ")
binding_site_list_strip = binding_site_list.replace(' ','')
mylist = binding_site_list_strip.split(',')
indexx = 0
numberz = ''
for number in mylist:
    if '-' not in mylist[indexx]:
        numberz += ', ' + mylist[indexx]
    if '-' in mylist[indexx]:
        indexxx = 0
        for everydigit in mylist[indexx]:
            if everydigit == '-':
                hyphen_pos = indexxx
            indexxx = indexxx + 1
        rnums_seq_script = mylist[indexx][0:hyphen_pos]
        num_counter = mylist[indexx][0:hyphen_pos]
        numberz += ', '+ num_counter
        for y in range(int(mylist[indexx][(hyphen_pos + 1):]) - int(mylist[indexx][0:hyphen_pos])):
            num_counter = str(int(num_counter) + 1)
            numberz += ', ' + str(num_counter)
    indexx = indexx + 1
tidied_output = numberz[2:]
rnums_script = "rnums : [" + tidied_output + "]"
#print rnums_script

These are the chains present in the structure:   ['A', 'C', 'B', 'D']
Would you like to display all chains (answer with yes or no)?   no
Type in the chain you would like to display:   B
Is there a binding or active site you would like to display (answer with yes or no)?   yes
Would you like to display the site on all chains (answer with yes or no)?   no
Which chain would you like to display the site on?   B
List the sequences of the active sites (ie 6, 7, 120-134):   25-60


## Only displaying chain B and only highlighting binding site on chain B

In [15]:
pdb_file_path3 = pdbl.retrieve_pdb_file(pdb_name)

Structure exists: '/Users/LAURENCE/Desktop/Senior Design/e0/pdb3e04.ent' 


In [17]:
PDBViewer_options(pdb_file_path3)